# PACKAGES

In [101]:
import os
import sys

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

import math
#from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from collections import Counter
import operator
#from rank_bm25 import BM25Okapi

# DATA PRE-PROCESSING

In [85]:
def parse_queries(txt):
    parsed_queries = []
    queries = re.findall(r'<top>(.*?)</top>', txt, re.DOTALL)
    counter = 1  # Initialize counter
    for q in queries:
        query = {}
      #  query['num']  = re.search(r'<num>(.*?)</num>', q).group(1).strip() if re.search(r'<num>(.*?)</num>', q) else None
        query['num'] = counter
        query['title'] = re.search(r'<title>(.*?)</title>', q, re.DOTALL).group(1).strip() if re.search(r'<title>(.*?)</title>', q, re.DOTALL) else None
        parsed_queries.append(query)
        counter += 1
    return parsed_queries

In [134]:
def read_queries(relative_path):
    all_data = []
    full_path = os.path.join(os.getcwd(), relative_path)
    for filename in os.listdir(full_path):
        with open(os.path.join(full_path, filename), 'r', encoding='utf-8') as file:
            data = parse_queries(file.read())
            all_data.extend(data)  # Assuming you want to collect data from all files
    return all_data

queries = read_queries('queries')
#print(queries)

In [3]:
def parse_docs(txt):
    docs = re.findall(r'<doc>(.*?)</doc>', txt, re.DOTALL)
    parsed_data = []
    for doc in docs:
        doc_data = {}
        doc_data['docno'] = re.search(r'<docno>(.*?)</docno>', doc).group(1).strip() if re.search(r'<docno>(.*?)</docno>', doc) else None
        doc_data['title'] = re.search(r'<title>(.*?)</title>', doc, re.DOTALL).group(1).strip() if re.search(r'<title>(.*?)</title>', doc, re.DOTALL) else None
        doc_data['author'] = re.search(r'<author>(.*?)</author>', doc).group(1).strip() if re.search(r'<author>(.*?)</author>', doc) else None
        #doc_data['bib'] = re.search(r'<bib>(.*?)</bib>', doc).group(1).strip() if re.search(r'<bib>(.*?)</bib>', doc) else None
        doc_data['text'] = re.search(r'<text>(.*?)</text>', doc, re.DOTALL).group(1).strip() if re.search(r'<text>(.*?)</text>', doc, re.DOTALL) else None
        parsed_data.append(doc_data)
    return parsed_data

In [135]:
def read_docs(relative_path):
    all_data = []
    full_path = os.path.join(os.getcwd(), relative_path)
    for filename in os.listdir(full_path):
        with open(os.path.join(full_path, filename), 'r', encoding='utf-8') as file:
            data = parse_docs(file.read())
            all_data.extend(data)  # Assuming you want to collect data from all files
    return all_data

docs = read_docs('docs')
#print(docs)

In [88]:
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()
def preprocess(text):
    # Tokenization and Lowercasing
    tokens = word_tokenize(text.lower())  
    # Remove punctuation, stopwords, and perform stemming
    processed_tokens = []
    for token in tokens:
        # Remove punctuation and check if token is not empty after stripping
        token = token.strip(string.punctuation)
        if token != '' and len(token) >= 2:
            # Perform stemming and filter out stopwords
            stemmed_token = porter.stem(token)
            if stemmed_token not in stop_words:
                processed_tokens.append(stemmed_token)  
    return processed_tokens  #return as string   #' '.join(processed_tokens)

In [89]:
###Check###
num_docs = len(docs)
print(num_docs)
num_queries = len(queries)
print(num_queries)

1400
225


# INVERTED INDEX

In [90]:
##Using list of dictionaries data structure
def create_inverted_index(data):
    inverted_index = defaultdict(list)
    for doc in data:
        doc_id = doc['docno']
        text = doc['text']
        terms = preprocess(text)
        term_positions = defaultdict(list)  
        # Populate term positions
        for position, term in enumerate(terms, start=1):
            term_positions[term].append(position)      
        # Populate inverted index
        for term, positions in term_positions.items():
            term_frequency = len(positions)  # Calculate term frequency
            posting = {"doc": doc_id, "term_frequency": term_frequency, "positions": positions}
            inverted_index[term].append(posting)
    return inverted_index

inverted_index = create_inverted_index(docs) # Create inverted index
# Save inverted index to a file to view
with open("inverted_index.txt", "w") as file:
    for term, postings in inverted_index.items():
        file.write(f"{term}: {postings}\n")

# TF-IDF 

In [91]:
def compute_tfidf(inverted_index, docs):
    tfidf = {}
    N = len(docs)  # Total number of documents
    # Compute document frequencies (df) for each term
    df = {term: len(postings) for term, postings in inverted_index.items()}   
    # Compute TF-IDF for each term-document pair
    for term, postings in inverted_index.items():
        idf = math.log(N / (df[term] + 1))  # Add 1 to avoid division by zero
        for posting in postings:
            doc_id = posting['doc']
            tf = posting['term_frequency']
            tfidf_score = tf * idf
            tfidf.setdefault(doc_id, {})[term] = tfidf_score 
    return tfidf

# Vector Space Model

In [92]:
# Compute Docs TF-IDF scores
docs_tfidf = compute_tfidf(inverted_index, docs)
# Compute document frequencies (df) for each term  (needed for query TF-IDF computation)
df = {term: len(postings) for term, postings in inverted_index.items()}
# Compute total number of documents (N)
N = len(docs)

In [93]:
def compute_query_tfidf(query_terms, docs_tfidf, df, N):
    query_tfidf = {}
    for term in query_terms:
        # Calculate IDF for term
        idf = math.log(N / (df.get(term, 0) + 1))
        tfidf = 1 * idf
        query_tfidf[term] = tfidf
    return query_tfidf

# Calculate cosine similarity (vec1 and vec2 are dictionaries)
def cosine_similarity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])    
    sum1 = sum([val ** 2 for val in vec1.values()])
    sum2 = sum([val ** 2 for val in vec2.values()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)    
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
    
def rank_vsm(query, docs_tfidf, df, N):
    preprocessed_query = preprocess(query['title'])
    query_tfidf = compute_query_tfidf(preprocessed_query, docs_tfidf, df, N)    
    scores = []
    for doc_id, doc_tfidf in docs_tfidf.items():
        score = cosine_similarity(query_tfidf, doc_tfidf)
        scores.append((doc_id, score))   
    ranked_docs = sorted(scores, key=lambda x: x[1], reverse=True)
    return ranked_docs

def evaluate(queries, docs_tfidf, df, N):
    results = []
    for query in queries:
        ranked_docs = rank_vsm(query, docs_tfidf, df, N)
        for doc_id, score in ranked_docs:
            results.append((query['num'], doc_id, score))
    return results

# Output Results

In [94]:
def write_results(results, file_path, run_name):
    with open(file_path, 'w') as f:
        rank = 1  # Initialize rank counter
        for result in results:
            if result[2] != 0.0:  # Exclude results with score 0.0
                query_id =  str(result[0])
                document_id =  str(result[1])
                score = result[2]
                # Write in TREC format: <query_id> <Q0> <doc_id> <rank> <score> <run_id>
                f.write(f"{query_id} Q0 {document_id} {rank} {score} {run_name}\n")
                rank += 1  # Increment rank for the next document
                

In [95]:
# Evaluate and Output VSM
vsm_results = evaluate(queries, docs_tfidf, df, N)
write_results(vsm_results, 'vsm_results.txt', 'VSM_Run')


# BM25 

In [96]:
# computing idf - N is total numbe of documents, doc_freq is document frequency of term
def idf(term, N, doc_freq):
    return math.log((N - doc_freq + 0.5) / (doc_freq + 0.5) + 1)


In [97]:
def compute_bm25(inverted_index, docs, k1=1.5, b=0.75):
    bm25_scores = {}
    N = len(docs)
    avgdl = sum(len(doc['text'].split()) for doc in docs) / N  # Average document length
    df = {term: len(postings) for term, postings in inverted_index.items()}  # Document frequency
    
    for doc in docs:
        doc_id = doc['docno']
        doc_len = len(doc['text'].split())
        bm25_scores[doc_id] = {}
        
        for term, postings in inverted_index.items():
            idf_val = idf(term, N, df[term])
            term_freq = sum(posting['term_frequency'] for posting in postings if posting['doc'] == doc_id)
            # Calculate BM25 term score
            term_score = idf_val * (term_freq * (k1 + 1)) / (term_freq + k1 * (1 - b + b * (doc_len / avgdl)))
            if term_score > 0:
                bm25_scores[doc_id][term] = term_score
    return bm25_scores

bm25_scores = compute_bm25(inverted_index, docs)

In [98]:
def rank_bm25(query, bm25_scores):
    query_terms = preprocess(query['title'])  # Assuming the query is in 'text' key
    doc_scores = {}

    for doc_id, scores in bm25_scores.items():
        doc_score = sum(scores.get(term, 0) for term in query_terms)
        if doc_score > 0:
            doc_scores[doc_id] = doc_score

    ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    return ranked_docs


In [99]:
def evaluate_bm25(bm25_scores, queries, docs):
    results = []
    for query in queries:
        ranked_docs = rank_bm25(query, bm25_scores)
        for doc_id, score in ranked_docs:
            # Note: Ensure your query objects have a 'num' field that uniquely identifies them
            results.append((query['num'], doc_id, score))
    return results


In [100]:
# Evaluate and Output BM25
evaluation_results = evaluate_bm25(bm25_scores, queries, docs)
write_results(evaluation_results, 'bm25_results.txt', 'BM25_Run')

# Language Model

In [132]:
### Query likelihood with Jelinek-Mercer smoothing ##
def compute_query_likelihood(query, inverted_index, total_terms, smoothing_param=0.1):
    query_terms = preprocess(query['title'])
    doc_likelihoods = {}
    collection_frequencies = {term: sum(posting['term_frequency'] for posting in postings) for term, postings in inverted_index.items()}
    total_collection_terms = sum(collection_frequencies.values())
    
    for term in query_terms: 
        if term in inverted_index:
            postings = inverted_index[term]
            for posting in postings:
                doc_id = posting['doc']
                term_freq = posting['term_frequency']
                doc_length = total_terms.get(doc_id, 0)
                term_prob = (1 - smoothing_param) * (term_freq / doc_length) + smoothing_param * (collection_frequencies.get(term, 0) / total_collection_terms)
                doc_likelihoods[doc_id] = doc_likelihoods.get(doc_id, 1) * term_prob
    
    return doc_likelihoods

#  rank documents based on query likelihood
def rank_qlm(query_likelihoods):
    return sorted(query_likelihoods.items(), key=lambda x: x[1], reverse=True)

# evaluate Query Likelihood Model
def evaluate_qlm(queries, inverted_index, total_terms, smoothing_param=0.1):
    results = []
    for query in queries:
        query_likelihoods = compute_query_likelihood(query, inverted_index, total_terms, smoothing_param)
        ranked_docs = rank_qlm(query_likelihoods)
        for doc_id, likelihood in ranked_docs:
            results.append((query['num'], doc_id, likelihood))
    return results

# Compute total number of terms per document (The Document Model)
total_terms = {}
for doc in docs:
    for term, postings in inverted_index.items():
        for posting in postings:
            if posting['doc'] == doc['docno']:
                total_terms[doc['docno']] = total_terms.get(doc['docno'], 0) + posting['term_frequency']

qlm_results = evaluate_qlm(queries, inverted_index, total_terms, smoothing_param=0.1)
# Write results to a file
write_results(qlm_results, 'QLM_JM_results.txt', 'QLM_JM_Run')


# EVALUATION - NDCG

In [133]:
def read_relevance_file():
    relevance_dict = {}
    file_name = 'cranqrel.trec.txt'
    with open(file_name, 'r') as file:
        for line in file:
            parts = line.strip().split()
            query_id = parts[0]
            doc_id = parts[2]
            relevance_score = int(parts[3])
            if query_id not in relevance_dict:
                relevance_dict[query_id] = {}
            relevance_dict[query_id][doc_id] = relevance_score
    return relevance_dict


def read_results_file(retrieval_method):
    results_dict = {}
    directory = 'outputs'
    file_name = f'{retrieval_method}_results.txt'
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            query_id = parts[0]
            doc_id = parts[2]
            rank = int(parts[3])
            if query_id not in results_dict:
                results_dict[query_id] = []
            results_dict[query_id].append(doc_id)
    return results_dict


def calculate_dcg(relevance_scores, k):
    dcg = 0.0
    for i in range(min(k, len(relevance_scores))):
        rel = relevance_scores[i]
        dcg += (2 ** rel - 1) / (i + 2)
    return dcg

def calculate_ndcg(relevance_judgments, retrieval_methods, k_values):
    ndcg_scores = {}
    for retrieval_method in retrieval_methods:
        ndcg_scores[retrieval_method] = {}
        for k in k_values:
            ndcg_total = 0.0
            query_count = 0
            search_results = read_results_file(retrieval_method)
            for query_id, relevant_docs in relevance_judgments.items():
                if query_id in search_results:
                    retrieved_docs = search_results[query_id]
                    relevance_scores = [relevant_docs.get(doc_id, 0) for doc_id in retrieved_docs]
                    idcg = calculate_dcg(sorted(relevance_scores, reverse=True), k)
                    dcg = calculate_dcg(relevance_scores, k)
                    ndcg = dcg / idcg if idcg > 0 else 0.0
                    ndcg_total += ndcg
                    query_count += 1
            ndcg_scores[retrieval_method][k] = ndcg_total / query_count if query_count > 0 else 0.0
    return ndcg_scores


# Read relevance judgments
relevance_judgments = read_relevance_file()

# Calculate NDCG@5 and NDCG@10 for BM25 and VSM
retrieval_methods = ['bm25', 'vsm', 'QLM_JM']
k_values = [5, 10]
ndcg_scores = calculate_ndcg(relevance_judgments, retrieval_methods, k_values)

# Print NDCG scores for each retrieval method and each value of k
for method, scores in ndcg_scores.items():
    print(f"Retrieval Method: {method}")
    for k, ndcg in scores.items():
        print(f"NDCG@{k}: {ndcg}")

Retrieval Method: bm25
NDCG@5: 0.3691850297076175
NDCG@10: 0.37526805466757485
Retrieval Method: vsm
NDCG@5: 0.35308069516546847
NDCG@10: 0.357273384482503
Retrieval Method: QLM_JM
NDCG@5: 0.053041299149673524
NDCG@10: 0.060392399501209115


# APPENDIX - models not used

In [103]:
## Regular Query Likelihood Model & Document Model (No Smoothing) ### 
def compute_query_likelihood(query, inverted_index, total_terms):
    query_terms = preprocess(query['title'])
    doc_likelihoods = {}
    for term in query_terms:
        if term in inverted_index:
            postings = inverted_index[term]
            for posting in postings:
                doc_id = posting['doc']
                term_freq = posting['term_frequency']
                doc_length = total_terms.get(doc_id, 0)
                likelihood = (term_freq / doc_length) if doc_length > 0 else 0
                doc_likelihoods[doc_id] = doc_likelihoods.get(doc_id, 1) * likelihood
    return doc_likelihoods

def rank_qlm(query_likelihoods):
    return sorted(query_likelihoods.items(), key=lambda x: x[1], reverse=True)

def evaluate_qlm(queries, inverted_index, total_terms):
    results = []
    for query in queries:
        query_likelihoods = compute_query_likelihood(query, inverted_index, total_terms)
        ranked_docs = rank_qlm(query_likelihoods)
        for doc_id, likelihood in ranked_docs:
            results.append((query['num'], doc_id, likelihood))
    return results

# Compute total number of terms per document (The Document Model)
total_terms = {}
for doc in docs:
    for term, postings in inverted_index.items():
        for posting in postings:
            if posting['doc'] == doc['docno']:
                total_terms[doc['docno']] = total_terms.get(doc['docno'], 0) + posting['term_frequency']
# Evaluate QLM
qlm_results = evaluate_qlm(queries, inverted_index, total_terms)

# Write QLM results to a file
write_results(qlm_results, 'QLM_results.txt', 'QLM_Run')


In [128]:
##Trying out both Jelinek-Mercer and Dirichlet smoothing
def compute_collection_freq(inverted_index):
    collection_freq = {}
    total_docs = len(inverted_index)
    for term, postings in inverted_index.items():
        collection_freq[term] = sum(posting['term_frequency'] for posting in postings) / total_docs
    return collection_freq

def compute_query_likelihood_jm(query, inverted_index, total_terms, collection_freq, smoothing_param):
    query_terms = preprocess(query['title'])
    doc_likelihoods = {}
    for term in query_terms:
        if term in inverted_index:
            postings = inverted_index[term]
            for posting in postings:
                doc_id = posting['doc']
                term_freq = posting['term_frequency']
                doc_length = total_terms.get(doc_id, 0)
                likelihood = (1 - smoothing_param) * (term_freq / doc_length) + smoothing_param * collection_freq.get(term, 0)
                doc_likelihoods[doc_id] = doc_likelihoods.get(doc_id, 1) * likelihood
    return doc_likelihoods

def compute_query_likelihood_dirichlet(query, inverted_index, total_terms, collection_freq, mu):
    query_terms = preprocess(query['title'])
    doc_likelihoods = {}
    collection_size = sum(total_terms.values())
    for term in query_terms:
        if term in inverted_index:
            postings = inverted_index[term]
            for posting in postings:
                doc_id = posting['doc']
                term_freq = posting['term_frequency']
                doc_length = total_terms.get(doc_id, 0)
                likelihood = (term_freq + mu * (collection_freq.get(term, 0) / collection_size)) / (doc_length + mu)
                doc_likelihoods[doc_id] = doc_likelihoods.get(doc_id, 1) * likelihood
    return doc_likelihoods

# Define the evaluate_query_model function to handle both Jelinek-Mercer and Dirichlet smoothing
def evaluate_query_model(queries, inverted_index, total_terms, collection_freq=None, smoothing_param=None, mu=None):
    results = []
    for query in queries:
        if smoothing_param is not None:
            query_likelihoods = compute_query_likelihood_jm(query, inverted_index, total_terms, collection_freq, smoothing_param)
        elif mu is not None:
            query_likelihoods = compute_query_likelihood_dirichlet(query, inverted_index, total_terms, collection_freq, mu)
        ranked_docs = rank_qlm(query_likelihoods)
        for doc_id, likelihood in ranked_docs:
            results.append((query['num'], doc_id, likelihood))
    return results

# Compute collection frequency
collection_freq = compute_collection_freq(inverted_index)

# Evaluate and Output results for Jelinek-Mercer Smoothing
smoothing_param = 0.1  # Set your chosen value for Jelinek-Mercer smoothing
jm_results = evaluate_query_model(queries, inverted_index, total_terms, collection_freq=collection_freq, smoothing_param=smoothing_param)
write_results(jm_results, 'QLM_JM_results.txt', f'QLM_JM_Run_{smoothing_param}')

# Evaluate and Output results for Dirichlet Smoothing
mu = 1000  # Set your chosen value for Dirichlet smoothing
dirichlet_results = evaluate_query_model(queries, inverted_index, total_terms, collection_freq=collection_freq, mu=mu)
write_results(dirichlet_results, 'QLM_D_results.txt', f'QOLM_D_Run_{mu}')


In [130]:
### KL Divergence ####

# preprocess text
def preprocess(text):
    return nltk.word_tokenize(text.lower())

# compute KL divergence
def kl_divergence(p, q):
    return sum(p[i] * math.log2(p[i] / q[i]) for i in range(len(p)))

# query likelihood with Jelinek-Mercer smoothing and KL divergence
def compute_query_likelihood(query, inverted_index, total_terms, smoothing_param=0.1):
    query_terms = preprocess(query['title'])
    doc_likelihoods = {}
    collection_frequencies = {term: sum(posting['term_frequency'] for posting in postings) for term, postings in inverted_index.items()}
    total_collection_terms = sum(collection_frequencies.values())
    for term in query_terms:
        if term in inverted_index:
            postings = inverted_index[term]
            for posting in postings:
                doc_id = posting['doc']
                term_freq = posting['term_frequency']
                doc_length = total_terms.get(doc_id, 0)
                term_prob = (1 - smoothing_param) * (term_freq / doc_length) + smoothing_param * (collection_frequencies.get(term, 0) / total_collection_terms)
                doc_likelihoods[doc_id] = doc_likelihoods.get(doc_id, 1) * term_prob 
    # Normalize probabilities
    total_prob = sum(doc_likelihoods.values())
    for doc_id in doc_likelihoods:
        doc_likelihoods[doc_id] /= total_prob   
    return doc_likelihoods

# rank documents based on query likelihood
def rank_qlm(query_likelihoods):
    return sorted(query_likelihoods.items(), key=lambda x: x[1], reverse=True)

# evaluate Query Likelihood Model with KL divergence
def evaluate_qlm(queries, inverted_index, total_terms, smoothing_param=0.1):
    results = []
    for query in queries:
        query_likelihoods = compute_query_likelihood(query, inverted_index, total_terms, smoothing_param)
        ranked_docs = rank_qlm(query_likelihoods)
        for doc_id, likelihood in ranked_docs:
            results.append((query['num'], doc_id, likelihood))
    return results

# Compute total number of terms per document (document model)
total_terms = {}
for doc in docs:
    for term, postings in inverted_index.items():
        for posting in postings:
            if posting['doc'] == doc['docno']:
                total_terms[doc['docno']] = total_terms.get(doc['docno'], 0) + posting['term_frequency']

# Evaluate QLM with Jelinek-Mercer smoothing
qlm_results = evaluate_qlm(queries, inverted_index, total_terms, smoothing_param=0.1)

# Write QLM results to a file
write_results(qlm_results, 'QLM_KLD_results.txt', 'QLM_KLD_Run')
